<a href="https://colab.research.google.com/github/od41/handson-lm/blob/main/Classification_with_gen_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# Load our model
 pipe = pipeline(
"text2text-generation",
model="google/flan-t5-small",
device="cuda:0"
 )